In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
187,ABW,North America,Aruba,2020-09-21,3551.0,91.0,72.143,23.0,0.0,0.714,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
453,AFG,Asia,Afghanistan,2020-09-21,39044.0,0.0,46.857,1441.0,0.0,3.000,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
637,AGO,Africa,Angola,2020-09-21,3991.0,90.0,93.714,152.0,8.0,2.857,...,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15,0.581


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-20,France,454334.0,0.0,31410.0,0.0,0.0,0.0
0,2020-09-20,China,90355.0,0.0,4742.0,0.0,0.0,0.0
0,2020-09-20,Italy,298222.0,0.0,35718.0,0.0,0.0,0.0
0,2020-09-20,Spain,648957.0,0.0,30569.0,0.0,0.0,0.0
0,2020-09-20,United States,6840251.0,0.0,200467.0,0.0,0.0,0.0
0,2020-09-20,World,31311161.0,0.0,966514.0,0.0,0.0,0.0
0,2020-09-20,United Kingdom,393549.0,0.0,41811.0,0.0,0.0,0.0
0,2020-09-20,Germany,273412.0,0.0,9396.0,0.0,0.0,0.0
0,2020-09-20,Iran,422263.0,0.0,24335.0,0.0,0.0,0.0
0,2020-09-20,Turkey,302736.0,0.0,7495.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-21,France,454334.0,453763.0,31410.0,31285.0,571.0,125.0
0,2020-09-21,China,90355.0,90369.0,4742.0,4737.0,-14.0,5.0
0,2020-09-21,Italy,298222.0,298156.0,35718.0,35707.0,66.0,11.0
0,2020-09-20,Spain,648957.0,640040.0,30569.0,30495.0,8917.0,74.0
0,2020-09-21,United States,6840251.0,6804814.0,200467.0,199509.0,35437.0,958.0
0,2020-09-21,World,31311161.0,31091469.0,966514.0,961352.0,219692.0,5162.0
0,2020-09-21,United Kingdom,393549.0,394257.0,41811.0,41777.0,-708.0,34.0
0,2020-09-21,Germany,273412.0,272337.0,9396.0,9386.0,1075.0,10.0
0,2020-09-21,Iran,422263.0,422140.0,24335.0,24301.0,123.0,34.0
0,2020-09-21,Turkey,302736.0,302867.0,7495.0,7506.0,-131.0,-11.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")